In [ ]:
import nest_asyncio
nest_asyncio.apply()

from llama_parse import LlamaParse

parser = LlamaParse(result_type="markdown", language="ko")

file_path = r"techreader_data\LLM_TechLibrary.pdf"
parsed_docs = parser.load_data(file_path=file_path)  # 이제 정상 실행됨

docs = [doc.to_langchain_format() for doc in parsed_docs]
print(docs[0].page_content[:500])  # 일부 미리보기

import os
from llama_parse import LlamaParse

# PDF 파서 초기화
parser = LlamaParse(
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="gemini-2.5-pro",
    vendor_multimodal_api_key=os.environ["GOOGLE_API_KEY"],
    result_type="markdown",
    parsing_mode="Unstructured",
    language="ko",
    parsing_instruction="""
     당신은 PDF 문서를 구조화된 Markdown으로 변환하는 파서입니다.
     
    가장 중요한 규칙:
    모든 텍스트는 가능한 모두 출력해주세요. 
    첫 페이지를 제외한 Tech Guide와 Tech Trend는 소제목이 아닙니다. 
    
    
    변환 규칙:
    0. 원문 텍스트는 가능한 한 모두 보존하세요. 
    1. 문서의 '주요 제목'은 반드시 `# 제목` 형식으로 추출하세요.
       - 제목 바로 아래 줄에 '저자 | 소속'이 있으면 `Author: 이름 | 소속`으로 출력하세요.
    2. 본문 내의 소제목은 `## 소제목`으로 변환하세요. 
       - 단, '# 1.' 같은 번호 형식, 첫 페이지를 제외한 영어 한 단어, 결론을 제외한 짧은 음절(예: 비추천 용도, 최적 용도, 주의점)은 소제목으로 간주하지 마세요.
    3. 제목/소제목 외의 일반 문단은 그냥 텍스트로 출력하세요. 특정 페이지에 일반 문단만 있어도 그대로 출력하세요. # 표시하지 마세요. 
    4. 일반 문단은 그냥 텍스트로 출력하되 • 표시로 시작하는 것도 그대로 출력해주세요.   
    5. 모든 출력은 순수한 Markdown 형식으로 작성하세요. 불필요한 설명, 번역, 해설은 절대 추가하지 마세요. 텍스트를 요약하지 말고 그대로 출력하세요. 
    """
)

# 파일 경로
file_path = r"techreader_data\LLM_TechLibrary.pdf"
# TechReader_gayoon\techreader_data\LLM_TechLibrary.pdf
# PDF → 파싱
parsed_docs = parser.load_data(file_path=file_path)

# LangChain Document 변환
docs = [doc.to_langchain_format() for doc in parsed_docs]

# Markdown 저장
file_root, _ = os.path.splitext(file_path)
output_file_path = file_root + "_parsed0903_gemini.md"

full_text = "\n\n".join([doc.page_content for doc in docs])

with open(output_file_path, "w", encoding="utf-8") as f:
    f.write(full_text)

print(f"✅ 파일 저장 완료: {output_file_path}")


import os
from langchain_text_splitters import MarkdownHeaderTextSplitter

# 1. 마크다운 파일 불러오기
with open("techreader_data/LLM_TechLibrary_parsed0903_gemini.md", "r", encoding="utf-8") as f:
    markdown_text = f.read()

# 2. 분할 기준 헤더 정의
headers_to_split_on = [
    ("#", "Header 1"),    # # 제목
    ("##", "Header 2"),   # ## 소제목
    ("###", "Header 3"),  # ### 하위 소제목
]

# 3. Splitter 초기화
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# 4. 문서 분할
md_header_splits = markdown_splitter.split_text(markdown_text)

# 5. 결과 출력
for i, doc in enumerate(md_header_splits[:10]):  # 앞에서 10개만 출력
    print(f"==== Chunk {i+1} ====")
    print(doc.page_content[:300])  # 앞부분 미리보기
    print("메타데이터:", doc.metadata)
    print()

import csv

with open("techreader_data/chunks_output.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["Chunk No", "Metadata", "Content"])
    for i, doc in enumerate(md_header_splits, start=1):
        writer.writerow([i, doc.metadata, doc.page_content.strip()])

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import os

# 1. OpenAI Embedding 초기화
embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])

# 2. md_header_splits (문서 리스트)를 벡터화 후 저장
vectorstore = FAISS.from_documents(md_header_splits, embeddings)

# 3. 저장
vectorstore.save_local("faiss_index")

print("✅ 벡터스토어 저장 완료: faiss_index 폴더 생성됨")


# 저장된 FAISS 인덱스 불러오기
new_vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

# 검색 
query = "교사 모델과 학생 모델의 관계에서 발생하는 보안 위험은 무엇인가요?"
docs = new_vectorstore.similarity_search(query, k=3)

for d in docs:
    print("📌", d.page_content[:300])
    print("메타데이터:", d.metadata)
    print("="*50)


import os
import google.generativeai as genai

# Google AI Studio에서 발급받은 API 키 설정
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Gemini 모델 초기화 (최신은 gemini-1.5-pro 또는 gemini-1.5-flash)
model = genai.GenerativeModel("gemini-2.5-pro")

def generate_questions(header1, header2=None):
    topic = header1 if header2 is None else f"{header1} - {header2}"
    prompt = f"""
    너는 AI 최신 기술 동향을 분석해 사내 엔지니어와 연구원이 빠르게 이해할 수 있도록
    핵심 쟁점을 질문 형태로 정리하는 역할이다.
    지금 다루는 문서는 Tech Library Top1에 선정된 21페이지짜리 리포트이며, 재직자 전용이다.
    너의 목표는 주제와 관련된 기술적 쟁점과 연구 방향을 드러내는 예상 질문을 생성하는 것이다.

    [요구사항]
    - 주제: {topic}
    - 예상 질문은 총 5개를 만들어라.
    - 질문은 학생용 단순 이해 차원이 아니라, 엔지니어/연구자가
      토론·실험·설계 단계에서 실제로 고민할 만한 '기술적 질문'이어야 한다.
    - 질문은 명확하고 구체적으로 작성하라.
    """

    response = model.generate_content(prompt)
    return response.text.strip().split("\n")


questions_dict = {}

for doc in md_header_splits:
    h1 = doc.metadata.get("Header 1")
    h2 = doc.metadata.get("Header 2")

    if (h1, h2) not in questions_dict:
        q_list = generate_questions(h1, h2)
        questions_dict[(h1, h2)] = q_list

# 확인
for (h1, h2), q_list in questions_dict.items():
    print(f"\n📌 {h1} > {h2 if h2 else ''}")
    for q in q_list:
        print(" -", q)


# 보고서 본문 기반 질문 리스트
content_questions_dict = {}

for doc in md_header_splits:
    h1 = doc.metadata.get("Header 1")
    h2 = doc.metadata.get("Header 2")
    content = doc.page_content

    if (h1, h2) not in content_questions_dict:
        q_list = generate_questions_from_content(h1, h2, content)
        content_questions_dict[(h1, h2)] = q_list

# 확인
for (h1, h2), q_list in content_questions_dict.items():
    print(f"\n📌 {h1} > {h2 if h2 else ''}")
    for q in q_list:
        print(" -", q)


# 답변도 같이 생성하기 
import csv

def load_questions(csv_path):
    questions = []
    with open(csv_path, "r", encoding="utf-8-sig") as f:
        reader = csv.DictReader(f)
        for row in reader:
            questions.append({
                "Header 1": row["Header 1"],
                "Header 2": row["Header 2"],
                "Question": row["Question"]
            })
    return questions

questions = load_questions("techreader_data/content_based_questions_clean.csv")
print(f"총 {len(questions)}개 질문 불러옴")

import google.generativeai as genai
import os, re

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-pro")

def clean_answer_text(text: str) -> str:
    # 불필요한 멘트 제거
    text = re.sub(r"(네, 알겠습니다.*|물론입니다.*|다음은.*|아래와 같이.*)", "", text)
    text = re.sub(r"[*#]{2,}", "", text)  # ###, *** 제거
    # 문장 끝의 ... → .
    text = re.sub(r"\.{2,}", ".", text.strip())
    return text.strip()

def generate_answer(question, header1, header2, content):
    prompt = f"""
    너는 AI 최신 기술 리포트를 분석하는 LLM 엔지니어다.
    문서 주제: {header1} - {header2 if header2 else ""}
    본문 내용 (발췌): {content[:3000] if content else ""}

    [요구사항]
    - 아래 질문에 대해 보고서 본문을 근거로 심층적인 답변을 작성하라.
    - 답변은 연구 보고서 스타일로 작성하며, 3~4문단으로 구성하라.
    - 전체 분량은 800~1000자 내외가 되도록 하라.
    - 각 문단은 완결된 문장으로 끝내라.
    - 서론(질문의 중요성), 본론(기술적 근거·세부 분석), 결론(핵심 요약과 시사점) 구조를 따르라.
    - 마지막 문장은 반드시 마침표 하나(.)로 끝내라. 불필요한 ... 은 쓰지 말라.
    - 출력은 반드시 '답변: ' 형식으로 하라.
    - 최종 답변은 반드시 마침표 하나(.)로 끝내라.
    - '...' 나 불필요한 반복 마침표는 절대 사용하지 말라.


    질문: {question}
    """

    try:
        response = model.generate_content(
            prompt,
            generation_config={"max_output_tokens": 9096, "temperature": 0.7}
        )

        if response.candidates and response.candidates[0].content.parts:
            answer = response.candidates[0].content.parts[0].text.strip()
        else:
            answer = "[⚠️ 답변 없음: 토큰 한도 초과 또는 안전 필터 차단]"

    except Exception as e:
        answer = f"[⚠️ 에러 발생: {str(e)}]"

    # 후처리: 결론 보강
    if not answer.endswith(("이다.", "있다.", "할 수 있다.")):
        try:
            fix_prompt = f"""
            다음 답변이 결론 없이 끝났습니다. 
            불필요한 멘트(예: '네, 알겠습니다', '물론입니다', '다음은', '### 결론', '***')는 쓰지 말고, 
            보고서 스타일의 결론 문단(3~4문장)을 작성하세요. 마지막 문장은 반드시 마침표 하나(.)로 끝내라.

            불완전 답변: {answer}
            """
            fix_response = model.generate_content(fix_prompt)
            if fix_response.candidates and fix_response.candidates[0].content.parts:
                answer += "\n\n" + fix_response.candidates[0].content.parts[0].text.strip()
        except:
            pass

    # 마지막 정리 (멘트/###/*** 제거, ... → .)
    return clean_answer_text(answer) or ""

for i, q in enumerate(questions):  # 전체 다 돌림
    h1, h2, question = q["Header 1"], q["Header 2"], q["Question"]

    # md_header_splits에서 헤더 매칭해서 본문 불러오기
    content = ""
    for doc in md_header_splits:
        if doc.metadata.get("Header 1") == h1 and doc.metadata.get("Header 2") == h2:
            content = doc.page_content
            break

    answer = generate_answer(question, h1, h2, content)
    q["Answer"] = answer  # 답변 추가

    print(f"\nQ{i+1}/{len(questions)}: {question}")
    print(f"A: {answer[:]}...")  # 앞부분만 미리보기 
    
    
    
import csv

output_path = "techreader_data/content_based_questions_with_answers.csv"

with open(output_path, "w", newline="", encoding="utf-8-sig") as f:
    fieldnames = ["Header 1", "Header 2", "Question", "Answer"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    for q in questions:
        writer.writerow({
            "Header 1": q["Header 1"],
            "Header 2": q["Header 2"],
            "Question": q["Question"],
            "Answer": q.get("Answer", "")
        })

print(f"✅ 최종 저장 완료: {output_path}")


import csv

def load_questions(csv_path):
    questions = []
    with open(csv_path, "r", encoding="utf-8-sig") as f:
        reader = csv.DictReader(f)
        for row in reader:
            questions.append({
                "Header 1": row["Header 1"],
                "Header 2": row["Header 2"],
                "Question": row["Question"]
            })
    return questions

questions = load_questions("techreader_data/header_based_questions_clean.csv")
print(f"총 {len(questions)}개 질문 불러옴")

import google.generativeai as genai
import os, re

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-pro")

def clean_answer_text(text: str) -> str:
    # 불필요한 멘트 제거
    text = re.sub(r"(네, 알겠습니다.*|물론입니다.*|다음은.*|아래와 같이.*)", "", text)
    text = re.sub(r"[*#]{2,}", "", text)  # ###, *** 제거
    # 문장 끝의 ... → .
    text = re.sub(r"\.{2,}", ".", text.strip())
    return text.strip()

def generate_answer(question, header1, header2, content):
    prompt = f"""
    너는 AI 최신 기술 리포트를 분석하는 LLM 엔지니어다.
    문서 주제: {header1} - {header2 if header2 else ""}
    본문 내용 (발췌): {content[:3000] if content else ""}

    [요구사항]
    - 아래 질문에 대해 보고서 본문을 근거로 심층적인 답변을 작성하라.
    - 답변은 연구 보고서 스타일로 작성하며, 3~4문단으로 구성하라.
    - 전체 분량은 800~1000자 내외가 되도록 하라.
    - 각 문단은 완결된 문장으로 끝내라.
    - 서론(질문의 중요성), 본론(기술적 근거·세부 분석), 결론(핵심 요약과 시사점) 구조를 따르라.
    - 마지막 문장은 반드시 마침표 하나(.)로 끝내라. 불필요한 ... 은 쓰지 말라.
    - 출력은 반드시 '답변: ' 형식으로 하라.
    - 최종 답변은 반드시 마침표 하나(.)로 끝내라.
    - '...' 나 불필요한 반복 마침표는 절대 사용하지 말라.


    질문: {question}
    """

    try:
        response = model.generate_content(
            prompt,
            generation_config={"max_output_tokens": 9096, "temperature": 0.7}
        )

        if response.candidates and response.candidates[0].content.parts:
            answer = response.candidates[0].content.parts[0].text.strip()
        else:
            answer = "[⚠️ 답변 없음: 토큰 한도 초과 또는 안전 필터 차단]"

    except Exception as e:
        answer = f"[⚠️ 에러 발생: {str(e)}]"

    # 후처리: 결론 보강
    if not answer.endswith(("이다.", "있다.", "할 수 있다.")):
        try:
            fix_prompt = f"""
            다음 답변이 결론 없이 끝났습니다. 
            불필요한 멘트(예: '네, 알겠습니다', '물론입니다', '다음은', '### 결론', '***')는 쓰지 말고, 
            보고서 스타일의 결론 문단(3~4문장)을 작성하세요. 마지막 문장은 반드시 마침표 하나(.)로 끝내라.

            불완전 답변: {answer}
            """
            fix_response = model.generate_content(fix_prompt)
            if fix_response.candidates and fix_response.candidates[0].content.parts:
                answer += "\n\n" + fix_response.candidates[0].content.parts[0].text.strip()
        except:
            pass

    # 마지막 정리 (멘트/###/*** 제거, ... → .)
    return clean_answer_text(answer) or ""

for i, q in enumerate(questions):  # 전체 다 돌림
    h1, h2, question = q["Header 1"], q["Header 2"], q["Question"]

    # md_header_splits에서 헤더 매칭해서 본문 불러오기
    content = ""
    for doc in md_header_splits:
        if doc.metadata.get("Header 1") == h1 and doc.metadata.get("Header 2") == h2:
            content = doc.page_content
            break

    answer = generate_answer(question, h1, h2, content)
    q["Answer"] = answer  # 답변 추가

    print(f"\nQ{i+1}/{len(questions)}: {question}")
    print(f"A: {answer[:]}...")  # 앞부분만 미리보기 
    
    
    
import csv

output_path = "techreader_data/header_based_questions_with_answers.csv"

with open(output_path, "w", newline="", encoding="utf-8-sig") as f:
    fieldnames = ["Header 1", "Header 2", "Question", "Answer"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    for q in questions:
        writer.writerow({
            "Header 1": q["Header 1"],
            "Header 2": q["Header 2"],
            "Question": q["Question"],
            "Answer": q.get("Answer", "")
        })

print(f"✅ 최종 저장 완료: {output_path}")

# 예상 질문 답변 쌍 저장 완료 (CSV 파일 2개) 


# 위의 paraphrase된 질문에는 질문을 포함한 광범위한 내용이 담겨 필터링 필요 - 우선 시범 테스트 진행 
import pandas as pd
import os
import google.generativeai as genai

# Gemini 초기화
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-pro")

def generate_paraphrases(question, n=5):
    prompt = f"""
    다음 질문을 의미가 동일하지만 표현이 다른 방식으로 {n}개 만들어줘.
    출력은 반드시 질문만, 각 줄 하나씩, 불필요한 설명이나 번호, 불릿, 마크다운 기호 없이 작성해.
    질문: {question}
    """
    try:
        response = model.generate_content(prompt)
        if response.candidates and response.candidates[0].content.parts:
            text = response.candidates[0].content.parts[0].text.strip()
            # 줄 단위 split
            lines = [line.strip(" -•0123456789.") for line in text.split("\n") if line.strip()]
            # "?" 로 끝나는 질문만 남김
            paras = [line for line in lines if line.endswith("?")]
            return paras
    except Exception as e:
        print(f"⚠️ 오류 발생: {e}")
    return []

# 원본 CSV 로드
input_path = "techreader_data/content_based_questions_with_answers.csv"
df = pd.read_csv(input_path)

# 상위 3개 질문만 테스트
sample_questions = df["Question"].head(3).tolist()

print("🔹 Paraphrase 생성 테스트 (3개 질문)\n")
for i, q in enumerate(sample_questions, start=1):
    paras = generate_paraphrases(q, n=3)
    print(f"Q{i}: {q}")
    for j, p in enumerate(paras, start=1):
        print(f"   → P{j}: {p}") 
    print("-" * 60)


import pandas as pd
import os
import google.generativeai as genai

# Gemini 초기화
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-pro")

def generate_paraphrases(question, n=4):
    prompt = f"""
    다음 질문을 의미가 동일하지만 표현이 다른 방식으로 {n}개 만들어줘.
    출력은 반드시 질문만, 각 줄 하나씩, 불필요한 설명이나 번호, 불릿, 마크다운 기호 없이 작성해.
    질문: {question}
    """
    try:
        response = model.generate_content(prompt)
        if response.candidates and response.candidates[0].content.parts:
            text = response.candidates[0].content.parts[0].text.strip()
            lines = [line.strip(" -•0123456789.") for line in text.split("\n") if line.strip()]
            paras = [line for line in lines if line.endswith("?")]  # 질문만
            return paras
    except Exception as e:
        print(f"⚠️ 오류 발생: {e}")
    return []

# 원본 CSV 로드
input_path = "techreader_data/content_based_questions_with_answers.csv"
df = pd.read_csv(input_path)

# Paraphrases 생성
df["Paraphrases"] = df["Question"].apply(lambda q: generate_paraphrases(q, n=4))

# 새 파일로 저장
output_path = "techreader_data/content_based_FAQ2_with_paraphrases.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"✅ Paraphrases 추가 완료: {output_path}")

# TechReader_gayoon/techreader_data/header_based_questions_with_answers.csv 

import pandas as pd
import os
import google.generativeai as genai

# Gemini 초기화
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-pro")

def generate_paraphrases(question, n=4):
    prompt = f"""
    다음 질문을 의미가 동일하지만 표현이 다른 방식으로 {n}개 만들어줘.
    출력은 반드시 질문만, 각 줄 하나씩, 불필요한 설명이나 번호, 불릿, 마크다운 기호 없이 작성해.
    질문: {question}
    """
    try:
        response = model.generate_content(prompt)
        if response.candidates and response.candidates[0].content.parts:
            text = response.candidates[0].content.parts[0].text.strip()
            lines = [line.strip(" -•0123456789.") for line in text.split("\n") if line.strip()]
            paras = [line for line in lines if line.endswith("?")]  # 질문만
            return paras
    except Exception as e:
        print(f"⚠️ 오류 발생: {e}")
    return []

# 원본 CSV 로드
input_path = "techreader_data/header_based_questions_with_answers.csv "
df = pd.read_csv(input_path)

# Paraphrases 생성
df["Paraphrases"] = df["Question"].apply(lambda q: generate_paraphrases(q, n=4))

# 새 파일로 저장
output_path = "techreader_data/header_based_FAQ2_with_paraphrases.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"✅ Paraphrases 추가 완료: {output_path}")


faq 질문 답변쌍은 CacheBackedEmbeddings 사용 
일반 질문 답변쌍은 업스테이지 임베딩 모델 사용 


import pandas as pd
import ast
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_upstage import UpstageEmbeddings
from langchain.storage import LocalFileStore
from pathlib import Path

embeddings = UpstageEmbeddings(model="solar-embedding-1-large", api_key="UPSTAGE_API_KEY")


# ----------------------------
# 1. FAQ 데이터 (CacheBackedEmbeddings)
# ----------------------------
def load_faq_docs(file_path):
    df = pd.read_csv(file_path)
    docs = []
    for _, row in df.iterrows():
        base_q = row["Question"]
        paras = []
        try:
            paras = ast.literal_eval(row["Paraphrases"])
        except:
            pass
        all_qs = [base_q] + paras
        for q in all_qs:
            docs.append(Document(
                page_content=q,
                metadata={"Answer": row["Answer"],
                          "Header 1": row.get("Header 1", ""),
                          "Header 2": row.get("Header 2", "")}
            ))
    return docs

faq_files = [
    "techreader_data/header_based_FAQ2_with_paraphrases.csv",
    "techreader_data/content_based_FAQ2_with_paraphrases.csv"
]

faq_docs = []
for f in faq_files:
    faq_docs.extend(load_faq_docs(f))

# 캐시 디렉토리
cache_dir = Path("techreader_data/faq_cache")
store = LocalFileStore(str(cache_dir))

# 기본 임베딩
base_embeddings = OpenAIEmbeddings()

# CacheBackedEmbeddings 구성
faq_embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=base_embeddings,
    document_embedding_cache=store
)

faq_db = FAISS.from_documents(faq_docs, faq_embeddings)
faq_db.save_local("techreader_data/faq_index")
faq_retriever = faq_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

print("✅ FAQ Retriever 준비 완료")

import pandas as pd
import ast
from langchain.schema import Document

chunks_df = pd.read_csv("techreader_data/chunks_output.csv")

chunk_docs = []
for _, row in chunks_df.iterrows():
    content = row["Content"]

    # Metadata 문자열 → dict 변환
    metadata = {}
    if isinstance(row["Metadata"], str) and row["Metadata"].strip() != "{}":
        try:
            metadata = ast.literal_eval(row["Metadata"])
        except Exception:
            metadata = {"raw_metadata": row["Metadata"]}

    # Chunk 번호도 추가
    metadata["Chunk No"] = row["Chunk No"]

    chunk_docs.append(Document(page_content=content, metadata=metadata))

print(f"✅ 총 {len(chunk_docs)}개 chunk 변환 완료")
print("예시 Document:", chunk_docs[0])

from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings  # pip install langchain-upstage

# Upstage 임베딩 초기화
chunk_embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large", 
    api_key=os.environ["UPSTAGE_API_KEY"]
)

# FAISS 인덱스 구축
chunk_db = FAISS.from_documents(chunk_docs, chunk_embeddings)

# 로컬 저장
chunk_db.save_local("techreader_data/chunk_index")

# Retriever
chunk_retriever = chunk_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

print("✅ Chunk Retriever 준비 완료")


query = "교사와 학생 모델의 차이점은 무엇인가?"

response = hybrid_search(query, faq_retriever, chunk_retriever)
print(response)















